# Whistle Detection with Continuous Kernel Convolutional Networks

In [ ]:
import sys

COLAB_RUNTIME = 'google.colab' in sys.modules
if COLAB_RUNTIME:
    !git init
    !git remote add origin https://github.com/neverorfrog/whistlenet.git
    !git pull origin main
    !pip install -q -r requirements.txt
else:
    !pip install -q -r requirements.txt
    !pre-commit install

## Config Loading

In [2]:
from config import *
import os
import warnings
warnings.filterwarnings("ignore")
import torch
import numpy as np
from whistlenet.core.utils import project_root

config_path = os.path.join(project_root(), "config","whistle_config.yaml")
config: Config = load_config(config_path)
torch_config: TorchConfig = config.torch
dataset_config: DatasetConfig = config.dataset
trainer_config: TrainerConfig = config.trainer
baseline_config: BaselineConfig = config.baseline
whistlenet_config: WhistlenetConfig = config.whistlenet
torch.manual_seed(torch_config.seed)
np.random.seed(torch_config.seed)

## Dataset Preparation

The dataset has already been uploaded to google drive. The labelling was executed by hand by using Audacity. The processing was done as such:
- Read every single .wav audio file with librosa
- Do frequency analysis
- Extract frequency associated amplitudes for every window

This is an example of frequency plot. The highlited windows are whistle samples.

<p align="left">
  <img src="../../docs/assets/image.png" alt="Alt text" width="300"/>
</p>

In [ ]:
from whistlenet.data import WhistleDataset
from whistlenet.core.utils import plot, NUM_FREQS
from whistlenet.core.utils.audio import SampleType
import gdown

download_path = os.path.join(dataset_config.download_folder, dataset_config.name)
gdown.download_folder(
   dataset_config.drive_url, output=download_path
)

dataset = WhistleDataset(dataset_config)
dataset.summarize()
reshaped = dataset.train_data.reshape((dataset.train_data.data.shape[0], NUM_FREQS))
plot(reshaped, dataset.train_data.labels, SampleType)

In [ ]:
from whistlenet.models import WhistleNet, Baseline
from whistlenet.core import LightningTrainer

model = WhistleNet(in_channels=1, out_channels=1, config=whistlenet_config)
trainer = LightningTrainer(trainer_config)
trainer.fit(model,dataset)
trainer.test(model, dataset)

In [ ]:
baseline = Baseline(config=baseline_config)
trainer = LightningTrainer(trainer_config)
trainer.fit(baseline,dataset)
trainer.test(baseline, dataset)